In [1]:
import os
import json
import re
import shutil
input_folder = "encoded_yosys"
output_folder = "done_KNN"
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [20]:
# Use list comprehension to create a list of file paths
verilog_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith('.txt')]

# Print the list of file paths
print(len(verilog_files))
print(type(verilog_files[0]))
print(verilog_files[0])


387
<class 'str'>
encoded_yosys\adder10_synth.txt


In [21]:
classes = ['adder', 'comparator', 'decoder', 'encoder','mult','mux','pe','sub','and','or','not','nand','nor','xnor', 'xor']
label_mapping = {
        'adder': 0, 'comparator': 1, 'decoder': 2,
        'encoder': 3, 'mult': 4, 'mux': 5, 'pe': 6, 'sub': 7, 'and': 8, 'or': 9, 'not': 10, 'nand': 11, 'nor': 12, 'xnor': 13,'xor':14
    }

In [22]:
ground_truth_labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13, 14] 
labels_torch = torch.tensor(ground_truth_labels)
one_hot_labels = F.one_hot(labels_torch, num_classes=15)

# type_index = {
#     "input":    "1000000",
#     "output":   "0100000",
#     "wire":     "0010000",
#     "not" :     "0001000",
#     "or" :      "0000100",
#     "and":      "0000010",
#     "constVal": "0000001"
# }

# 0         1          2       3       4       5       6
# input     output     wire    not     or      and     constVal


In [23]:
verilog_files_info = []
labels = []
labels_nums = []
for file_path in verilog_files:
    file_info = []
    
    input_count = 0
    output_count = 0
    wire_count = 0
    not_count = 0
    or_count = 0
    and_count = 0
    constVal_count = 0
    
    with open(file_path, "r") as file:
        st = file_path.split("\\")[1].split(".")[0].split("_")[0]

        matches = [string for string in label_mapping.keys() if re.match(f"^({string})\d+", st)]
        # print(file_path)
        # print(st)
        # print(matches)
        labels_nums.append(label_mapping[matches[0]])
        # print(labels_nums)
        y = one_hot_labels[label_mapping[matches[0]]].tolist()
        labels.append(y)
        loaded_data = json.load(file)
        
        for sublist in loaded_data[0]:
            if sublist[0]: # input
                input_count +=1
                
            elif sublist[1]: #output
                output_count += 1

            elif sublist[2]: #wire
                wire_count +=1
                
            elif sublist[3]:
                not_count+=1
                
            elif sublist[4]:
                or_count+=1

            elif sublist[5]:
                and_count+=1

            elif sublist[6]:
                constVal_count+=1
                

            else: 
                print("eh el habal da")
        file_info.extend([input_count, output_count, wire_count, not_count, or_count, and_count, constVal_count])
    verilog_files_info.append(file_info)

verilog_files_info

[[3, 2, 23, 6, 10, 10, 0],
 [3, 2, 9, 5, 4, 4, 2],
 [2, 2, 2, 1, 1, 2, 0],
 [3, 2, 22, 6, 11, 8, 0],
 [2, 2, 2, 1, 1, 2, 0],
 [2, 2, 2, 1, 1, 2, 0],
 [3, 2, 46, 12, 20, 19, 0],
 [3, 2, 39, 9, 18, 17, 0],
 [3, 2, 46, 12, 20, 19, 0],
 [3, 2, 39, 9, 17, 18, 0],
 [3, 2, 10, 3, 4, 5, 0],
 [3, 2, 37, 9, 17, 16, 0],
 [3, 2, 46, 12, 20, 19, 0],
 [3, 2, 72, 16, 30, 35, 0],
 [3, 2, 149, 34, 57, 75, 0],
 [3, 2, 36, 8, 15, 18, 0],
 [3, 2, 42, 11, 18, 18, 0],
 [3, 2, 42, 11, 18, 18, 0],
 [3, 2, 406, 99, 165, 175, 0],
 [3, 1, 313, 71, 138, 136, 0],
 [3, 2, 10, 3, 5, 4, 0],
 [3, 2, 10, 3, 5, 4, 0],
 [3, 2, 7, 1, 4, 4, 0],
 [3, 2, 10, 3, 4, 5, 0],
 [3, 2, 10, 3, 5, 4, 0],
 [3, 2, 46, 11, 18, 22, 0],
 [2, 2, 4, 5, 1, 2, 2],
 [5, 2, 23, 6, 10, 10, 0],
 [2, 1, 0, 0, 0, 1, 0],
 [2, 1, 0, 0, 0, 4, 0],
 [2, 1, 0, 0, 0, 4, 0],
 [2, 1, 0, 0, 0, 4, 0],
 [2, 1, 0, 0, 0, 3, 0],
 [2, 1, 0, 0, 0, 8, 0],
 [2, 1, 0, 0, 0, 1, 0],
 [2, 1, 0, 0, 0, 8, 0],
 [2, 1, 0, 0, 0, 16, 0],
 [2, 1, 0, 0, 0, 2, 0],
 [2, 1, 0, 0, 0

In [24]:
# len(verilog_files_info)
# len(labels)
verilog_files_info = np.array(verilog_files_info)
verilog_files_info.shape

(387, 7)

In [25]:
scaler = MinMaxScaler()

X = np.array(verilog_files_info)
X_normalized = scaler.fit_transform(X)
# print(type(X_normalized))
labels = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(X_normalized, labels, test_size=0.3, random_state=42)

# print(X_train)
# print(y_train)
knn = KNeighborsClassifier(n_neighbors=5)  # You can choose the number of neighbors
knn.fit(X_train, y_train)

knn_y_pred = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_y_pred)
print("KNN Accuracy:", knn_accuracy)

KNN Accuracy: 0.5128205128205128


In [27]:
import joblib
joblib.dump(knn, 'knn_model_files_51.pkl')

KNN_loaded_model = joblib.load('knn_model_files_51.pkl')

In [28]:
from sklearn import svm

scaler = MinMaxScaler()

X = np.array(verilog_files_info)
X_normalized = scaler.fit_transform(X)
labels_nums = np.array(labels_nums)

X_train, X_test, y_train, y_test = train_test_split(X_normalized, labels_nums, test_size=0.3, random_state=42)

svm_model = svm.SVC(kernel='linear')  # Using a linear kernel for simplicity

svm_model.fit(X_train, y_train)


y_pred = svm_model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.06837606837606838
